In [4]:
import pandas as pd

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
df = pd.read_json('script-bag-of-words.json')

In [3]:
df.tail()

,episodeAlt,seasonNum,episodeNum,episodeTitle,text
68,S8E2,8,2,A Knight of the Seven Kingdoms,"[{'name': 'Daenerys Targaryen', 'text': 'About..."
69,S8E3,8,3,The Long Night,"[{'name': 'Northman #1', 'text': 'Oi!'}, {'nam..."
70,S8E4,8,4,The Last of the Starks,"[{'name': 'Jon Snow', 'text': 'And Everyone It..."
71,S8E5,8,5,The Bells,"[{'name': 'Lord Varys', 'text': 'And? Come Not..."
72,S8E6,8,6,The Iron Throne,"[{'name': 'Tyrion Lannister', 'text': 'I'll fi..."


In [4]:
dialogue = {}
for index, row in df.iterrows():
    for item in row['text']:
        if item['name'] in dialogue:
            #append
            dialogue[item['name']] = dialogue[item['name']] + item['text']
        else:
            #create character
            dialogue[item['name']] = item['text'] + " "

In [5]:
len(dialogue)

817

In [6]:
new_df = pd.DataFrame()
new_df['character'] = dialogue.keys()
new_df['words'] = dialogue.values()

In [7]:
new_df.iloc[:, 0:3].head()

,character,words
0,Will,"Easy, boy. I've I've Wildlings a a do ever in ..."
1,Waymar Royce,One They're What a and another before d'you ea...
2,Gared,Wall. We back head should the to Our They We W...
3,Jon Snow,Father's Go on. watching. And mother. yourBran...
4,Septa Mordane,"Fine Well always. as done. work, I Quite beaut..."


In [9]:
new_df.shape

(817, 2)

In [10]:
new_df['num_words'] = new_df['words'].apply(lambda x:len(x.split()))

In [12]:
new_df = new_df.sort_values('num_words', ascending=False)

In [13]:
new_df = new_df.head(100)

In [14]:
new_df.shape

(100, 3)

In [15]:
new_df.head()

,character,words,num_words
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [17]:
embeddings = cv.fit_transform(new_df['words']).toarray()

In [18]:
embeddings = embeddings.astype('float')

In [19]:
from sklearn.manifold import TSNE

In [26]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

In [30]:
from sklearn.utils import parallel_backend
with parallel_backend('loky'):
    tsne = TSNE(n_components=2, verbose=1, random_state=123)
    z=tsne.fit_transform(embeddings)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 100 samples in 0.015s...


AttributeError: 'NoneType' object has no attribute 'split'

In [29]:
new_df['x'] = z.T[0]
new_df['y'] = z.T[1]

NameError: name 'z' is not defined

In [ ]:
import plotly.express as px
fig = px.scatter(new_df.head(25), x="x", y="y", color="character")
fig.show()